In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# import 2nd dataset from lloyds
bank = pd.read_csv('simulated_transaction_2024.csv')

In [25]:
bank.tail(10)

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
230586,06/12/2023,20:39,166019891.0,9482.471051,-62.47,NaN,AMAZON
230587,06/12/2023,20:40,975345315.0,40907.910578,-73.71,NaN,Tesco
230588,06/12/2023,20:43,106601471.0,11920.401863,-37.25,NaN,Coop Local
230589,06/12/2023,20:48,522188082.0,8471.141437,-17.05,NaN,AMAZON
230590,06/12/2023,20:50,256002321.0,7582.389937,-24.80,NaN,Sainsbury Local
230591,06/12/2023,20:54,581655972.0,45935.206861,-41.06,NaN,Tesco
230592,06/12/2023,20:55,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local
230593,06/12/2023,21:05,824916823.0,9709.172159,-32.94,NaN,Deliveroo
230594,06/12/2023,21:13,366550080.0,26834.165794,-19.25,NaN,Amazon
230595,06/12/2023,21:32,259015404.0,1145.349837,-37.64,NaN,JustEat


## Investigate Categories

In [9]:
merchants = bank['Third Party Name'].unique()
print(merchants)

# print number of unique merchants
print('Number of unique merchants: ', len(merchants))

['Westport Care Home' 'Barbiee Boutique' nan 'Fat Face' 'Lavender Primary'
 'A Cut Above' 'Tesco' 'Brilliant Brushes' 'Costa Coffee'
 'Premier Finance' 'North Face' 'Craftastic' 'Fitted Stitch' 'Selfridges'
 'Starbucks' 'The Crown' 'Cass Art' 'Green Park Academy' 'Rose & Crown'
 'Kings Arms' 'Sunny Care Nursery' 'A Yarn Story' 'Pets Corner'
 'Lavender Fields' 'Victoria Park' 'Halifax' 'LBG' 'Blizzard' 'Xbox'
 'Mojang Studios' 'PureGym' 'Disney' 'Netflix' 'Grand Union BJJ' 'Amazon'
 'SquareOnix' 'Deliveroo' 'JustEat' 'Coop Local' 'Sainsbury' 'AMAZON'
 'The Works' "Blackwell's" 'Topshop' 'Matalan' 'Foyles' 'Wool'
 'Hobby Lobby' 'Revella' 'Sainsbury Local' 'Loosely Fitted'
 'Stitch By Stitch' 'Coffee #1' 'Hobbycraft' 'Lloyds Pharmacy'
 'Gamestation' 'CeX' 'Etsy' 'Five Senses Art' 'Sports Direct' 'Boots'
 'Reebok' 'JD Sports' 'Head' 'Frankie & Bennies' 'Gap Kids'
 'University College Hospital' 'CPA' 'Happy Days Home' 'Pets at Home'
 'Jollyes' 'Collector Cave' 'Vision Express' 'Green Park'


- Got roughly the same amount of merchants as in the last dataset but with actual business names.

#### Pricing distributions across similar merchants - supermarkets

In [29]:
# drop transactions where 'Amount' is positive as these are not spending out of the account
merchant_spending = bank[bank['Amount'] < 0]

# keep rows where 'Third Party Account No' is null as these aren't payments to personal accounts
merchant_spending = merchant_spending[merchant_spending['Third Party Account No'].isnull()]


merchant_spending.head(10)

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
495,01/01/2023,00:00,355733816.0,526.00,-664.0,NaN,Halifax
496,01/01/2023,00:00,472213568.0,2412.00,-600.0,NaN,LBG
497,01/01/2023,00:00,624500124.0,2572.00,-654.0,NaN,LBG
498,01/01/2023,00:00,203466392.0,3935.66,-672.0,NaN,Halifax
499,01/01/2023,00:00,768271776.0,2208.00,-632.0,NaN,LBG
500,01/01/2023,00:00,564744955.0,938.00,-572.0,NaN,Halifax
501,01/01/2023,00:00,675806859.0,452.00,-648.0,NaN,LBG
502,01/01/2023,00:00,456221621.0,2167.00,-664.0,NaN,Halifax
503,01/01/2023,00:00,350921975.0,1333.50,-665.0,NaN,Halifax
504,01/01/2023,00:00,331450202.0,776.00,-631.0,NaN,LBG


In [30]:
# select only the transactions that are from the merchant 'Tesco' and 'Sainsbury Local'
tesco = merchant_spending[merchant_spending['Third Party Name'] == 'Tesco']
sainsbury_local = merchant_spending[merchant_spending['Third Party Name'] == 'Sainsbury Local']
sainsbury = merchant_spending[merchant_spending['Third Party Name'] == 'Sainsbury']

supermarkets = pd.concat([sainsbury_local['Amount'].describe(), tesco['Amount'].describe(), sainsbury['Amount'].describe()],axis=1)
supermarkets.columns = ['Sainsbury Local', 'Tesco', 'Sainsbury']
supermarkets

,Sainsbury Local,Tesco,Sainsbury
count,3155.000000,33905.000000,3117.000000
mean,-42.450051,-75.041815,-43.679734
std,42.938658,50.944684,43.357148
min,-320.750000,-348.350000,-303.800000
25%,-57.905000,-102.440000,-60.280000
50%,-31.120000,-62.980000,-31.310000
75%,-11.935000,-38.270000,-11.970000
max,-0.010000,-0.010000,-0.010000


#### Pricing distributions over coffee shops

In [31]:
starbucks = merchant_spending[merchant_spending['Third Party Name'] == 'Starbucks']
costa_coffee = merchant_spending[merchant_spending['Third Party Name'] == 'Costa Coffee']
coffee_1 = merchant_spending[merchant_spending['Third Party Name'] == 'Coffee #1']

coffee = pd.concat([starbucks['Amount'].describe(), costa_coffee['Amount'].describe(), coffee_1['Amount'].describe()],axis=1)
coffee.columns = ['Starbucks', 'Costa Coffee', 'Coffee #1']
coffee

,Starbucks,Costa Coffee,Coffee #1
count,4821.000000,677.000000,2298.000000
mean,-5.984505,-5.988774,-5.995735
std,1.509033,0.945160,1.785399
min,-90.000000,-8.300000,-80.000000
25%,-6.500000,-6.500000,-6.500000
50%,-5.800000,-5.800000,-5.750000
75%,-5.400000,-5.350000,-5.400000
max,-2.800000,-2.800000,-2.800000


- Beginning to see rough alignment in terms of spending mean and variance

## Investigate Missing Values